In [3]:
### 协程实例

def consumer():
    r = ''
    while True:
        n = yield r
        if not n:   # return nothing when first start the generator
            return
        print('[CONSUMER] Consuming %s...' % n)
        r = '200 OK'

def produce(c):
    c.send(None)  # 必须有这一步用于产生generator
    n = 0
    while n < 5:
        n = n + 1
        print('[PRODUCER] Producing %s...' % n)
        r = c.send(n)
        print('[PRODUCER] Consumer return: %s' % r)
    c.close()

c = consumer()
produce(c)

[PRODUCER] Producing 1...
[CONSUMER] Consuming 1...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 2...
[CONSUMER] Consuming 2...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 3...
[CONSUMER] Consuming 3...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 4...
[CONSUMER] Consuming 4...
[PRODUCER] Consumer return: 200 OK
[PRODUCER] Producing 5...
[CONSUMER] Consuming 5...
[PRODUCER] Consumer return: 200 OK


In [4]:
### interprocess experiments 1 子进程不能改变主进程中全局变量的值

import multiprocessing
import time
import os
 
datalist=['+++'] #全局变量，主进程与子进程是并发执行的，他们不能共享全局变量(子进程不能改变主进程中全局变量的值)
 
def adddata():
    global datalist
    datalist.append(1)
    datalist.append(2)
    datalist.append(3)
    print("子进程",os.getpid(),datalist)
 
 
if __name__=="__main__":
 
    p=multiprocessing.Process(target=adddata,args=())
    p.start()
    p.join()
    datalist.append("a")
    datalist.append("b")
    datalist.append("c")
    print("主进程",os.getpid(),datalist)


子进程 10462 ['+++', 1, 2, 3]
主进程 10407 ['+++', 'a', 'b', 'c']


In [2]:
### 子进程修改全局变量数值
import multiprocessing
 
def  func(num):
    num.value=10.78  #子进程改变数值的值，主进程跟着改变
 
if  __name__=="__main__":
    num=multiprocessing.Value("d",10.0) # d表示数值,主进程与子进程共享这个value。（主进程与子进程都是用的同一个value）
    print(num.value)
 
    p=multiprocessing.Process(target=func,args=(num,))
    p.start()
    p.join()
 
    print(num.value)


10.0
10.78


In [10]:
### 子进程修改全局变量   数组
import multiprocessing
 
def  func(num):
    num[5]=9999   #子进程改变数组，主进程跟着改变
 
if  __name__=="__main__":
    num=multiprocessing.Array("i",[1,2,3,4,5,6])   #主进程与子进程共享这个数组
    print(num[:])
 
    p=multiprocessing.Process(target=func,args=(num,))
    p.start() 
    p.join()
 
    print(num[:])


[1, 2, 3, 4, 5, 6]
[1, 2, 3, 4, 5, 9999]


In [1]:
### 子进程修改全局变量   list and dict

import multiprocessing
 
def func(mydict,mylist):
    mydict["index1"]="aaaaaa"   #子进程改变dict,主进程跟着改变
    J={"idx1":3,"idx2":"bbbb"} # not available
    mylist.append(11)        #子进程改变List,主进程跟着改变
    mylist.append(22)
    mylist.append(33)
    
if __name__=="__main__":
#    with multiprocessing.Manager() as MG:   #重命名
        MG=multiprocessing.Manager()
        mydict=MG.dict()   #主进程与子进程共享这个字典
        mylist=MG.list()   #主进程与子进程共享这个List

        print(mylist)
        print(mydict)
        p=multiprocessing.Process(target=func,args=(mydict,mylist))
        p.start()
        p.join()
 
        print(mylist)
        print(mydict)


[]
{}
[11, 22, 33]
{'index1': 'aaaaaa'}


In [1]:
import socket

In [7]:
mydict.values()

['aaaaaa']

In [5]:
import multiprocessing
mg = multiprocessing.Manager().dict()

In [12]:
J=mydict.copy()

In [31]:
print(mydict.values())

['aaaaaa']


In [3]:
num.value

10.78

In [20]:
J.values()

dict_values(['aaaaaa'])

In [8]:
J={"a":1,"b":2}

In [9]:
J1=dict()

In [14]:
for s in J:
    J1[s]=J[s]

In [15]:
J1

{'a': 1, 'b': 2}

In [17]:
from multiprocessing import Process, Manager
import os
 
 
def f(d, l):
    d[os.getpid()] = os.getpid()
    l.append(os.getpid())
    print(d)
 
if __name__ == '__main__':
    with Manager() as manager:
        d = manager.dict()   # 声明一个字典，这个字典是用manger声明的，不是用dict()声明的
        # manger.dict()是用专门的语法生产一个可在多进程之间进行传递和共享的一个字典
        l = manager.list(range(5))  # 同样声明一个列表
        p_list = []
        for i in range(10):
            p = Process(target=f, args=(d, l))
            p.start()
            p_list.append(p)
        for res in p_list:
            res.join()
        print(d)
        print(l)

{2725: 2725}
{2725: 2725, 2728: 2728}
{2725: 2725, 2728: 2728, 2733: 2733}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748, 2757: 2757}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748, 2757: 2757, 2769: 2769}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748, 2757: 2757, 2769: 2769, 2781: 2781}
{2725: 2725, 2728: 2728, 2733: 2733, 2741: 2741, 2754: 2754, 2751: 2751, 2748: 2748, 2757: 2757, 2769: 2769, 2781: 2781}
[0, 1, 2, 3, 4, 2725, 2728, 2733, 2741, 2754, 2751, 2748, 2757, 2769, 2781]
